In [1]:
import pandas as pd
import numpy as np

import uuid
from collections import defaultdict
import datetime

In [2]:
SUM_INTERVALS = [(-7,0), (-14,0), (-30,0)]

## Generate some random data

In [3]:
uids=np.array([str(uuid.uuid4()) for _ in range(10)])
times = pd.date_range('2016-01-01','2016-02-01', freq='s')
data = dict(
    id=np.random.choice(uids, 100),
    timestamp=np.random.choice(times, 100),
    feature_a=np.ones(100),
    feature_b=np.zeros(100),
)


## Challenge:
The `SUM_INTERVAL` variable contains relative time intervals in days. So the first means 7 days back until today (asof time writing this 2016-02-01).
the dictionary `data` contains 10 distinct users which made visists over the timespan of one month. Each visit has a value for `feature a`, as awell as `feature_b` assigned to it. **For each user calculate the sum of it's respective features for each time interval. The final output should be a dataframe or numpy matrix containing one row per user, an id column an the feature columns (N_users, 1 + N_intervals*N_features)**

*It is encouraged to use the pandas library for this task but it is not required.*

In [4]:
def bin_sum_features(data, feature_names, today=pd.Timestamp('2016-02-01')):
    totals = defaultdict(int)
    
    for feature_name in feature_names:
        for i, interval in enumerate(SUM_INTERVALS):
            before = today + pd.Timedelta(days=interval[0])
            after = today + pd.Timedelta(days=interval[1])
            
            for value, timestamp in zip(data[feature_name], data['timestamp']):
                if  before <= timestamp and timestamp <= after:
                    totals['{}_{}'.format(feature_name, i)] += value
                
    return totals

In [5]:
df = pd.DataFrame.from_dict(data)
feature_names = ['feature_a', 'feature_b']

df.groupby('id').apply(bin_sum_features, feature_names=feature_names).apply(pd.Series).fillna(0)


,feature_a_0,feature_a_1,feature_a_2,feature_b_0,feature_b_1,feature_b_2
id,,,,,,
162258e3-bc29-4c9b-8cd8-f5bfb4cd16f1,1.0,3.0,12.0,0.0,0.0,0.0
1851d89a-f396-4c4c-ac3d-047be6620c2f,4.0,8.0,11.0,0.0,0.0,0.0
39edb864-ce89-4b26-9a6d-59da30a50677,4.0,5.0,8.0,0.0,0.0,0.0
3b234248-9de1-4437-8f86-673fb8fa44c9,2.0,3.0,11.0,0.0,0.0,0.0
4ff533d1-0c36-4014-9fd3-aaf05681e52d,3.0,6.0,12.0,0.0,0.0,0.0
88f12e10-89f1-4c60-a97b-da65db613700,0.0,1.0,9.0,0.0,0.0,0.0
8aafb00b-92f5-45b3-84d7-fa086ae32ec1,1.0,1.0,9.0,0.0,0.0,0.0
d35a5ea4-75ca-4ae2-9188-c94c1125435f,1.0,5.0,9.0,0.0,0.0,0.0
e2f5b0fc-c9f2-4d05-b394-b00cdc75ed32,2.0,6.0,8.0,0.0,0.0,0.0


# A few comments

- The result above does not follow the naming convention of the result below. This is because it would be a problem if
SUM_INTERVALS = [(-7,0), (-7,-1)]. Instead it is using the interval ids.
- As always with dates, hours and time zone will be a problem but I did not consider this here
- You'll notice 3 consecutive for loops. While it looks ugly, I did this because we only have to group + loop over all the lines of the dataframe once, hence it should be faster
- If there is a lot more data, bin_sum_features could be made faster by using an array instead of defaultdict. However it would be less readable

## Result shape example
below you see how the results could look like with pandas

In [6]:
pd.DataFrame([['1aa9204b-5956-41a3-96b6-58cbf6bc147e',1,2,3,4,5,6]], 
             columns=['id', 'feature_a_7', 'feature_a_14', 'feature_a_30', 
                      'feature_b_7', 'feature_b_14', 'feature_b_30'])

,id,feature_a_7,feature_a_14,feature_a_30,feature_b_7,feature_b_14,feature_b_30
0,1aa9204b-5956-41a3-96b6-58cbf6bc147e,1,2,3,4,5,6
